In [1]:
import os

os.chdir('..')

In [2]:
_config = {
    "num_epochs": 80,
    "preprocess_2d": "DepthposeNormalize2D",
    "preprocess_3d": "SplitToRelativeAbsAndMeanNormalize3D",
    "shuffle": True,
    "ordered_batch": False,
    # training
    "optimiser": "adam",
    "adam_amsgrad": True,
    "learning_rate": 1e-3,
    "sgd_momentum": 0,
    "batch_size": 1024,
    "train_time_flip": True,
    "test_time_flip": True,
    "lr_scheduler": {
        "type": "multiplicative",
        "multiplier": 0.95,
        "step_size": 1,
    },
    # dataset
    "train_data": "mpii_train",
    "pose2d_type": "hrnet",
    "pose3d_scaling": "normal",
    "megadepth_type": "megadepth_at_hrnet",
    "cap_25fps": True,
    "stride": 2,
    "simple_aug": True,  # augments data by duplicating each frame
}

In [3]:
import os
import pandas as pd
import torch
import h5py
import numpy as np
from scipy.spatial.transform import Rotation
from torch.utils.data import Dataset
from ai import cs

from databases import mupots_3d, mpii_3dhp, muco_temp
from databases.joint_sets import CocoExJoints, OpenPoseJoints, MuPoTSJoints
from databases.datasets import *
from training.loaders import ChunkedGenerator

In [4]:
joint_set = MuPoTSJoints()
connected_joints = joint_set.LIMBGRAPH

In [5]:
train_data = Mpi3dTrainDataset(
    _config["pose2d_type"],
    _config["pose3d_scaling"],
    _config["cap_25fps"],
    _config["stride"],
)

In [6]:
train_loader = ChunkedGenerator(
    train_data,
    _config["batch_size"],
    10,
    _config["train_time_flip"],
    shuffle=_config["shuffle"],
    ordered_batch=_config["ordered_batch"],
)

In [11]:
i = 0
d = train_data.prepare_sample([0,1,2])
nd = d['pose3d']
td = torch.from_numpy(nd)
# l = getbonelength(td, connected_joints)

In [12]:
nd.shape

(3, 17, 3)

In [16]:
from ai import cs
cj = np.array(connected_joints)
diff = nd[:,cj[:,0], :] - nd[:, cj[:, 1], :] # vec1 - vec2
r, t, p = cs.cart2sp(diff[:, :, 0], diff[:, :, 1], diff[:, :, 2])

In [20]:
def th_sp2cart(r, t, p):
    x = r*torch.cos(p)*torch.cos(t)
    y = r*torch.sin(p)*torch.cos(t)
    z = r*torch.sin(t)
    return x, y, z

def sp2cart(r, t, p):
    x = r*np.cos(p)*np.cos(t)
    y = r*np.sin(p)*np.cos(t)
    z = r*np.sin(t)
    return x, y, z

In [21]:
cj_index = [2, 1, 0, 5, 4, 3, 9, 8, 12, 11, 10, 15, 14, 13, 7, 6]
ordered_cj = cj[cj_index, :]

In [22]:
x, y, z = sp2cart(r, t, p)
# th_sp2cart(torch.from_numpy(r), torch.from_numpy(t), torch.from_numpy(p))
edges = np.stack((x, y, z), axis=-1)

In [23]:
edges.shape

(3, 16, 3)

In [24]:
res = np.zeros((3, 17, 3))
res[:, 14, :] = nd[:, 14, :]
for (a, b), i in zip(ordered_cj, cj_index):
    res[:, a, :] = res[:, b, :] + edges[:, i, :]

In [25]:
res.shape

(3, 17, 3)

In [ ]:
cs.sp2cart(r, t, p)

In [ ]:
x = []
y = []
z = []
j = []
for i, (f, t) in enumerate(connected_joints):
    x.append(nd[0, f, 0])
    y.append(nd[0, f, 1])
    z.append(nd[0, f, 2])
    x.append(nd[0, t, 0])
    y.append(nd[0, t, 1])
    z.append(nd[0, t, 2])
    j.append(i)
    j.append(i)

In [ ]:
df = pd.DataFrame({'x': x, 'y': y, 'z': z, 'j': j})

In [ ]:
import plotly.express as px
# df = px.data.iris()
# fig = px.scatter_3d(df, x='x', y='y', z='z')
fig = px.line_3d(df, x="x", y="y", z="z", color='j', title='Original')
fig.show()